In [12]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
# import pyLDAvis
# import pyLDAvis.gensim  # don't skip this
# import matplotlib.pyplot as plt
# %matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import nltk
import pickle
from langdetect import detect

In [13]:
from nltk.corpus import stopwords
stop_words = nltk.corpus.stopwords.words('english')

#stop_words =  list(spacy.lang.en.stop_words.STOP_WORDS)
stop_words.extend(['game','pokemon','pokémon'])

# for word in stop_words:
#     if word in stop_words: 
#         continue
#     else: stop_words.append(word)

# for word in stop_words:
#     no_punct = remove_punctuation(word)
#     if no_punct not in stop_words: 
#         stop_words.append(no_punct)

In [14]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [15]:
with open('pokemon.pickle','rb') as read_file:
    df = pickle.load(read_file)

df['rating'] = df['rating'].astype(int)

def sentiment(x):
    if x > 7:
        return 'positive'
    if x < 5:
        return 'negative'
    else: return 'mixed'

df['sentiment'] = df['rating'].apply(lambda x:sentiment(x))

df.reset_index(inplace = True, drop = True)

df.drop_duplicates(subset='name', keep = 'first', inplace = True)

def language_detection(x): 
    result = detect(x)
    if result == 'en':
        return x 
    else: return np.NaN 
    
df['review'] = df['review'].apply(lambda x:language_detection(x))

df.dropna(inplace = True)

def remove_r(x):
    return x.replace('\r','')

df['review'] = df['review'].apply(lambda x:remove_r(x))  

def strip_whitespace(x):
    return x.strip()

df['review'] = df['review'].apply(lambda x:strip_whitespace(x))  

In [16]:
negative = df[df['sentiment'] == 'negative'] 

In [17]:
negative

,name,date,rating,review,game,sentiment
0,ChocolateCrane,"Nov 18, 2019",4,Let us address the elephant in the room first....,sword,negative
3,firstlovezombie,"Nov 15, 2019",2,"I really love Pokemon, which is why I'm so cri...",sword,negative
4,FilthyActs88,"Nov 15, 2019",1,Pokémon sword and shield is a game of pure med...,sword,negative
5,Ninjasuite,"Nov 15, 2019",3,This is what happen if a company realize that ...,sword,negative
6,JustrzBustrz,"Nov 15, 2019",2,It’s an amazing feat how soulless this game fe...,sword,negative
...,...,...,...,...,...,...
2889,Gonz04,"Nov 20, 2019",0,"This game is so disappointing, I literally end...",shield,negative
2891,InTheMood,"Nov 20, 2019",1,This is by far the worst Pokémon game I have e...,shield,negative
2893,Dreadwolf85,"Nov 20, 2019",3,Lazy game design. Pokemon still dont hit each ...,shield,negative
2897,Cotroneo,"Nov 20, 2019",2,If you consider this game a successor to the s...,shield,negative


In [18]:
data = negative['review'].values.tolist()

In [19]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['let', 'us', 'address', 'the', 'elephant', 'in', 'the', 'room', 'first', 'the', 'absence', 'of', 'the', 'national', 'dex', 'yes', 'it', 'is', 'terrible', 'lazy', 'blah', 'blah', 'blah', 'am', 'sure', 'you', 've', 'already', 'heard', 'this', 'in', 'hundreds', 'of', 'other', 'reviews', 'so', 'instead', 'of', 'me', 'wasting', 'your', 'time', 'by', 'complaining', 'about', 'something', 'you', 'are', 'already', 'well', 'aware', 'of', 'will', 'focus', 'on', 'the', 'rest', 'of', 'swsh', 'shortcomings', 'for', 'reference', 'have', 'played', 'every', 'mainline', 'pokemon', 'game', 'to', 'completion', 'and', 'have', 'currently', 'sunk', 'hours', 'into', 'shield', 'imagine', 'my', 'criticisms', 'also', 'apply', 'to', 'sword', 'in', 'the', 'interest', 'of', 'brevity', 'will', 'be', 'utilizing', 'acronyms', 'swsh', 'sword', 'shield', 'sun', 'moon', 'etc', 'the', 'most', 'succinct', 'way', 'to', 'put', 'it', 'is', 'swsh', 'essentially', 'feels', 'like', 'ds', 'game', 'ported', 'to', 'the', 'switch'

In [20]:
len(data_words)

1231

In [36]:
# Build the bigram and trigram models - reference: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

bigram = gensim.models.Phrases(data_words, min_count=5, threshold=3) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=3)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example


In [39]:
print(trigram_mod[bigram_mod[data_words[7]]])

['pokemon_sword_and_shield', 'boast', 'expansive', 'landscapes', 'and', 'sense_of', 'ambition', 'at_first', 'the', 'more', 'the_game', 'goes_on', 'you', 'become', 'aware', 'of', 'how', 'hastily', 'put', 'together', 'and', 'hollow', 'it', 'really', 'is', 'the_main', 'selling_point', 'the_wild_area_is', 'smaller_than', 'you_would_expect', 'and', 'plain', 'ugly', 'to', 'boot', 'there_is', 'not', 'lot', 'to_do', 'there', 'the_same', 'games', 'for', 'the_game', 'traditional', 'routes', 'which_are', 'some_of_the', 'smallest', 'and', 'most', 'linear', 'yet', 'this', 'combined', 'with', 'the_game', 'interrupting', 'you', 'every', 'seconds', 'for', 'leon', 'to', 'brag', 'about', 'his', 'charizard', 'or', 'whatever', 'make', 'the_game', 'chore', 'to', 'sit', 'through', 'despite', 'their', 'short', 'length', 'that', 'not', 'to_say', 'the_game', 'doesn', 'have', 'it', 'positives', 'though', 'galar', 'can_be', 'quite', 'pretty', 'when_it_comes_to', 'towns', 'and', 'colors', 'and', 'there_are_some',

In [22]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [23]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

KeyboardInterrupt: 

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

In [15]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics = 4)

In [16]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.010*"bad" + 0.010*"animation" + 0.008*"game" + 0.008*"graphic" + '
  '0.007*"get" + 0.007*"good" + 0.007*"even" + 0.006*"make" + 0.006*"cut" + '
  '0.006*"move"'),
 (1,
  '0.010*"bad" + 0.009*"make" + 0.008*"animation" + 0.008*"game" + 0.007*"go" '
  '+ 0.006*"look" + 0.006*"battle" + 0.006*"also" + 0.006*"new" + '
  '0.005*"play"'),
 (2,
  '0.014*"game" + 0.012*"make" + 0.011*"bad" + 0.009*"even" + '
  '0.007*"animation" + 0.007*"new" + 0.007*"story" + 0.006*"would" + '
  '0.006*"graphic" + 0.006*"feel"'),
 (3,
  '0.016*"game" + 0.012*"make" + 0.010*"even" + 0.009*"new" + 0.008*"cut" + '
  '0.008*"animation" + 0.008*"play" + 0.007*"really" + 0.007*"good" + '
  '0.007*"bad"')]
